**TP 4 Apprentissage multi-niveau de bruits, Unet, Mnist**


**Exercice 1:** (conditionnement par le bruit) 
Construire un réseau de neurones `pot` (MLP) $\mathbb{R}^3 \rightarrow \mathbb{R}$ et `grad` (MLP) $\mathbb{R}^3 \rightarrow \mathbb{R}^2$ qui vont servir à apprendre  respectivement la densité (potentiel) et le score (son gradient) conditionnés par une valeur de bruit. 

In [ ]:
import torch
from torch import nn        #neuronal network
from torch import optim     #descente de gradient
from sklearn.model_selection import train_test_split        #split into train and test set
from torch.utils.data import DataLoader
import itertools
import numpy as np

In [ ]:
input_dim = 3       #input layer with 3 neurons
hidden_dim = 10     #hidden layer with 10 neurons
output_dim = 2

In [ ]:
## on ajoute un paramètre, car pot et grad sont maintenant de R^3 au lieu de R^2.
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(NeuralNetwork, self).__init__()
        self.dropout = nn.Dropout(0.2)
        self.layer_1 = nn.Linear(input_dim, hidden_dim)
        nn.init.kaiming_uniform_(self.layer_1.weight, nonlinearity="relu")
        self.layer_2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        self.dropout = nn.Dropout(0.2)
        #prend max(0,x)
        x = torch.nn.functional.relu(self.layer_1(x))
        self.dropout = nn.Dropout(0.2)
        #limite les amplitudes à 1. pour l'instant pas besoin
        x = torch.nn.functional.sigmoid(self.layer_2(x))

        return x

In [ ]:
#instantiate a model of NN class for density estimation
model = NeuralNetwork(input_dim, hidden_dim, output_dim)
print(model)

#alpha = step-size in direction of gradient
learning_rate = 0.1
#loss function giving error between real and estimated data
loss_fn = nn.MSELoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

**Exercice 2:** (apprentissage multi-bruit du score)
En s'inspirant de l'exercice 6 du TP 3 et en basant sur la méthode vue en cours, apprendre à un réseau de neurones à prédire l'inverse du bruit qui est rajouté à un point des données, conditionné par le niveau de bruit appliqué. Faire une représentation du champs de vecteur associé

In [ ]:
from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np
from torch.distributions.multivariate_normal import MultivariateNormal

In [ ]:
def deux_bosses(mean_1, mean_2, cov_1, cov_2, n):
    data_1 = np.random.multivariate_normal(mean_1, cov_1, size=n//2)
    data_2 = np.random.multivariate_normal(mean_2, cov_2, size=n//2)
    return np.concatenate((data_1, data_2))

In [ ]:
samples = deux_bosses([0, 0], [6, 4], [[1, 0], [0, 1]], [[1, 0], [0, 1]], 20000)
print(samples.shape)
plt.scatter(samples[:,0], samples[:,1])

In [ ]:
#split dataset in train and test

X_train, X_test = train_test_split(samples,test_size=.33, random_state=26)

In [ ]:
#enables to create different batches instead of treating whole dataset taking too much CPU

# Convert data to torch tensors
class Data():
    def __init__(self, X):
        self.X = torch.from_numpy(X.astype(np.float32))
        self.len = self.X.shape[0]

    def __getitem__(self, index):
        return self.X[index]

    def __len__(self):
        return self.len

In [ ]:
#set batch size for first model
batch_size = 100

# Instantiate training and test data
train_data = Data(X_train)
train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

test_data = Data(X_test)
test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)

# Check it's working
for batch, X in enumerate(train_dataloader):
    print(f"Batch: {batch+1}")
    print(f"X shape: {X.shape}")
    break

In [ ]:
mean = torch.Tensor([0,0])
cov2 = torch.Tensor([[1, 0], [0, 1]])
distrib = MultivariateNormal(loc=mean, covariance_matrix=cov2)
distrib.rsample(sample_shape=torch.Size([100, 2]))

In [ ]:
# 2. model pour grads

input_dim_grad = 3       #input layer with 2 neurons
hidden_dim_grad = 10     #hidden layer with 10 neurons
output_dim_grad = 2      #output layer with 1 neuron

class NeuralNetworkGrad(nn.Module):
    def __init__(self, input_dim_grad, hidden_dim_grad, output_dim_grad):
        super(NeuralNetworkGrad, self).__init__()
        self.dropout = nn.Dropout(0.2)
        self.layer_1 = nn.Linear(input_dim_grad, hidden_dim_grad)
        self.layer_2 = nn.Linear(hidden_dim_grad, output_dim_grad)

    def forward(self, x):
        #prend max(0,x)
        x = torch.nn.functional.relu(self.layer_1(x))
        x = self.dropout(x) #apply dropout correctly
        #do not use the sigmoid activation function for the second layer anymore.
        #gradients can be positive or negative -> they do not take values only between 0 and 1!
        #sigmoid prevents the model from learning correct gradients with negative values.
        x = self.layer_2(x)

        return x

#instantiate a model of NN class
modelGrad = NeuralNetworkGrad(input_dim_grad, hidden_dim_grad, output_dim_grad)
print(modelGrad)

In [ ]:
#alpha = step-size in direction of gradient
learning_rate = 0.01
#loss function giving error between real and estimated data
loss_fn = nn.MSELoss()

optimizer_grad = torch.optim.SGD(modelGrad.parameters(), lr=learning_rate)

In [ ]:
#compute average
sampav=np.average(samples, axis=0)

#compute maximal distance between average and all points in datasets
def maxdistance(dataset):
    #compute average
    average=np.average(samples, axis=0)
    dist = np.linalg.norm(average - dataset[0])
    for tup in range(len(dataset)):
        disttemp =np.linalg.norm(average-dataset[tup])
        if disttemp >dist:
            dist = disttemp
    return dist


#compute sigma_t
def sigmat(dataset,t):
    sigmaMin=0.01
    maxdist = maxdistance(dataset)
    sigmaMax = 2*maxdist
    return sigmaMin*((sigmaMax/sigmaMin)**t)
def sample_batch(dataset,n):
    sigmas=[]
    times=np.random.rand(n)

    for i in range(0,len(times)):
        sigmas.append(sigmat(dataset,times[i]))
    return torch.Tensor(sigmas)


print(sample_batch(samples,10))

In [ ]:
#Train the model
num_epochs = 1
loss_values_grad = []

for epoch in range(num_epochs):
    for X in train_dataloader:

        # zero the parameter gradients

        optimizer_grad.zero_grad()


        # forward + backward + optimize
        #on sample un batch de niveau de bruit
        batch_amplitude_bruit=sample_batch(samples,torch.Size([X.shape[0]])[0])
        #reparametrization trick

        bruit=distrib.rsample(sample_shape=torch.Size([X.shape[0]]))
        print(bruit.shape)
        print(batch_amplitude_bruit.shape)
        bruit=bruit*batch_amplitude_bruit[:,None]

        print(bruit.shape)
        pred = modelGrad(X+bruit)
        loss = loss_fn(pred,-bruit)
        loss_values_grad.append(loss.item())
        loss.backward()
        optimizer_grad.step()

print("Training Complete")

In [ ]:
def torch_ls(fon, p, n, epsi):
    point = p
    tensor = point
    mu1=torch.Tensor([[epsi, 0],[0, epsi]])
    mean = torch.Tensor([0,0])
    distrib = MultivariateNormal(loc=mean, covariance_matrix=mu1)
    for i in range(n):
        point = point + fon(point) + distrib.rsample(sample_shape=torch.Size([point.shape[0]]))
        print(point)
        print(tensor)
        tensor=torch.cat((tensor,point))

    return tensor

'''
def torch_ls2(fon, n):
    mu1=torch.Tensor([[5, 0],[0, 5]])
    depart=torch.Tensor(np.random.multivariate_normal([2.5,2.5],mu1,n)) #l'echantilloner d'une loi normale.
    #print(depart)
    liste= [torch_ls(fon,depart[i],30,0.01)[-1] for i in range(depart.shape[0])]
    return np.array(liste)
'''

liste2=torch_ls(modelGrad,torch.Tensor([[0,4],[2,3]]),100,0.01)
print(liste2.shape)

**Exercice 3:** (champ de vecteur en grande dimension)
Nous souhaitons appliquer la méthode ci-dessus à des images $28\times 28$ (dataset mnist). Nous voulons donc pour cela utiliser une architecture Unet. Construire un réseau de neurone `gradU` (Unet) $\mathbb{R}^{28\times 28} \rightarrow \mathbb{R}^{28\times 28}$

In [ ]:
m= nn.Conv2d(1, 4, kernel_size=4, padding=0, stride=2)

input = torch.randn(1,28,28)
output = m(input)
#output.shape

m2=nn.Conv2d(4, 8, kernel_size=4, padding=0, stride=2)
output2=m2(output)
#output2.shape

m3=nn.ConvTranspose2d(8, 4, kernel_size=4, stride=2)
m4= nn.ConvTranspose2d(4, 1, kernel_size=4, stride=2)

output3=m3(output2)
output4=m4(output3)
#output3.shape
#output4.shape

In [ ]:
class UNet_simple(nn.Module):
    def __init__(self, channels=[32, 64, 128, 256], embed_dim=256,
               text_dim=256):
        super().__init__()

        # Other model properties
        self.act = nn.SiLU()

        # Encoding layers
        self.conv1 = nn.Conv2d(1, channels[0], 3, stride=1, bias=False)
        self.gnorm1 = nn.GroupNorm(4, num_channels=channels[0])

        self.conv2 = nn.Conv2d(channels[0], channels[1], 3, stride=2, bias=False)

        self.gnorm2 = nn.GroupNorm(32, num_channels=channels[1])

        self.conv3 = nn.Conv2d(channels[1], channels[2], 3, stride=2, bias=False)
        self.gnorm3 = nn.GroupNorm(32, num_channels=channels[2])


        self.conv4 = nn.Conv2d(channels[2], channels[3], 3, stride=2, bias=False)
        self.gnorm4 = nn.GroupNorm(32, num_channels=channels[3])


        # Decoding layers
        self.tconv4 = nn.ConvTranspose2d(channels[3], channels[2], 3, stride=2, bias=False)
        self.tgnorm4 = nn.GroupNorm(32, num_channels=channels[2])

        self.tconv3 = nn.ConvTranspose2d(channels[2], channels[1], 3, stride=2, bias=False, output_padding=1)

        self.tgnorm3 = nn.GroupNorm(32, num_channels=channels[1])

        self.tconv2 = nn.ConvTranspose2d(channels[1], channels[0], 3, stride=2, bias=False, output_padding=1)

        self.tgnorm2 = nn.GroupNorm(32, num_channels=channels[0])

        self.tconv1 = nn.ConvTranspose2d(channels[0], 1, 3, stride=1)


    def forward(self, x):

        # Encoding
        h1 = self.act(self.gnorm1(self.conv1(x) ))
        h2 = self.act(self.gnorm2(self.conv2(h1) ))
        h3 = self.act(self.gnorm3(self.conv3(h2) ))
        h4 = self.act(self.gnorm4(self.conv4(h3) ))


        # Decoding
        h = self.act(self.tgnorm4(self.tconv4(h4) ))
        h = self.act(self.tgnorm3(self.tconv3(h + h3) ))
        h = self.act(self.tgnorm2(self.tconv2(h + h2) ))
        h = self.tconv1(h + h1)

        return h

modelGradU = UNet_simple()

**Exercice 4:** (estimation naive du score)
En se basant sur la méthode vue à l'exercice 6 du TP 3, apprendre au réseau `gradU` à prédire l'inverse du bruit qui est rajouté à un point des données mnist.

In [ ]:
import torchvision
from torchvision import transforms
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

batch_size = 4
trainset = torchvision.datasets.MNIST(root='./data/', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data/', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

In [ ]:
for X in trainloader:
  modelGradU(X[0])
  print(modelGradU(X[0]).shape)
  break

In [ ]:
#alpha = step-size in direction of gradient
learning_rate = 0.01
#loss function giving error between real and estimated data
loss_fn = nn.MSELoss()

optimizer_gradU = torch.optim.SGD(modelGradU.parameters(), lr=learning_rate)

meanU = torch.zeros(28*28)
covU = torch.eye(28*28)
distribU = MultivariateNormal(loc=meanU, covariance_matrix=covU)
distribU.rsample(sample_shape=torch.Size([100, 28*28]))

#Train the model
num_epochs = 1
loss_values_grad = []

for epoch in range(num_epochs):
    for X in trainloader:
        X=X[0]
        #print(torch.Size([X.shape[0]])[0])
        # zero the parameter gradients

        optimizer_gradU.zero_grad()


        # forward + backward + optimize
        #on sample un batch de niveau de bruit
        batch_amplitude_bruit=sample_batch(samples,torch.Size([X.shape[0]])[0])
        #print(batch_amplitude_bruit)
        #reparametrization trick

        bruit=distrib.rsample(sample_shape=torch.Size([X.shape[0]]))
        #print(X.shape[0])
        #print(bruit.shape)
        #print(batch_amplitude_bruit.shape)
        bruit=bruit*batch_amplitude_bruit[:,None]

        #print(bruit.shape)
        #print(X.shape)
        #print(torch.add(X,bruit.view(4,2,1,1)).shape)
        #print(bruit)
        #print(X)
        #print(torch.add(X,bruit.view(4,2,1,1)))
        pred = modelGradU(torch.add(X,bruit.view(4,2,1,1))[:, :1, :, :]) #torch.add to add bruit for every batch to all of 28x28 pixels.
                                                                         #have 8 values in bruit, thus 2 Channels, then keep only the first Channel with [:, :1, :, :]? otw. doesn't work in UNet.
        loss = loss_fn(pred,-bruit.view(4,2,1,1))     #need to correclty "view" bruit as 4,2,1,1 tensor.
        loss_values_grad.append(loss.item())
        loss.backward()
        optimizer_gradU.step()

print("Training Complete")

**Exercice 5:** (inférence par dynamique langevienne et visualisation)
En s'inspirant de l'exercice 3 du TP 3, écrire une fonction `langevin_sample(fon,n)` qui prend en paramètre une fonction `fon`$:\mathbb{R}^{28\times 28}\rightarrow\mathbb{R}^{28\times 28}$ et échantillone un élément suivant cette densité en échantillon un point de $\mathbb{R}^{28\times 28}$ selon une loi normale puis effectue n pas à l'aide du schéma numérique d'Euler, en ajoutant à chaque pas une petite perturbation tirée selon une loi normale. Faire une matrice avec en ligne 10 points dans la courbe intégrale d'un échantillon et en colonne les courbes intégralles de 10 échantillons.

In [ ]:
## coder ici

**Exercice 6:** (conditionnement par le bruit) 
Construire un réseau de neurones `gradUmulti` (Unet) $\mathbb{R}^{28\times 28+1} \rightarrow \mathbb{R}^{28\times 28}$ qui va servir à apprendre  le score conditionné par une valeur de bruit. 

In [2]:
## coder ici

**Exercice 7:** (apprentissage multi-bruit du score)
En s'inspirant de l'exercice 2, apprendre au réseau `gradUmulti` à prédire l'inverse du bruit qui est rajouté à un point des données, conditionné par le niveau de bruit appliqué.

In [ ]:
## coder ici

**Exercice 8:** (inférence par dynamique langevienne)
En s'inspirant de l'exercice 5, écrire une fonction `langevin_sample_schedule(fon,n)` qui prend en paramètre une fonction `fon`$:\mathbb{R}^{28\times 28+1}\rightarrow\mathbb{R}^{28\times 28}$ et échantillone un élément suivant la distribution des données. Pour ce faire, elle échantillonne un point de $\mathbb{R}^{28\times 28}$ selon une loi normale puis effectue n pas à l'aide du schéma numérique d'Euler, en ajoutant à chaque pas une petite perturbation tirée selon une loi normale, en prenant soin de de choisir le niveau de bruit en fonction d'un scheduler. Sampler et représenter 10 courbes intégrales.

In [3]:
## coder ici